In [1]:
import os
import glob
import shutil

# Primeiro é necessario separar as imagens do dataset em labels e images, pois neste
# dataset todas vieram juntas no mesmo diretorio
# Alem disso tiramos o _anno.bmp do final das mascaras

base_dir = 'dataset/original-dataset'

images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

bmp_files = glob.glob(os.path.join(base_dir, "*.bmp"))
for bmp_path in bmp_files:
    file_name = os.path.basename(bmp_path)

    if file_name.endswith("_anno.bmp"):
        dest = os.path.join(labels_dir, file_name.replace('_anno.bmp','.bmp'))
    else:
        dest = os.path.join(images_dir, file_name)

    shutil.copy2(bmp_path, dest)

print(f"{len(glob.glob(os.path.join(images_dir, '*.bmp')))} imagens em 'images/'")
print(f"{len(glob.glob(os.path.join(labels_dir, '*.bmp')))} máscaras em 'labels/'")

165 imagens em 'images/'
165 máscaras em 'labels/'


In [1]:
import albumentations as A
import os
import random
import numpy as np
import config
import sys
import cv2
sys.path.append('../util')
from DatasetAugmentation import *


#Insira o nome do root do dataset original
original_dataset_path = './dataset/original-dataset'
output_base           = config.dataset_path

# -----------------------------
# Parâmetros
# -----------------------------
N = 15  # número de aumentações
num_to_valid = 24    # número de imagens a mover do train para valid
num_to_test  = 25    # número de imagens a mover do train para test


target_size  = (256, 256)
random.seed(42)

# -----------------------------
# Caminhos de entrada e saída
# -----------------------------
orig_train_img_dir  = os.path.join(original_dataset_path, 'images')
orig_train_mask_dir = os.path.join(original_dataset_path, 'labels')
orig_valid_img_dir  = os.path.join(original_dataset_path, 'blank')
orig_valid_mask_dir = os.path.join(original_dataset_path, 'blank')
orig_test_img_dir   = os.path.join(original_dataset_path, 'blank')
orig_test_mask_dir  = os.path.join(original_dataset_path, 'blank')



output_dirs = {
    'train_images': os.path.join(output_base, 'images/train'),
    'train_labels': os.path.join(output_base, 'labels/train'),
    'valid_images': os.path.join(output_base, 'images/valid'),
    'valid_labels': os.path.join(output_base, 'labels/valid'),
    'test_images':  os.path.join(output_base, 'images/test'),
    'test_labels':  os.path.join(output_base, 'labels/test'),
}

transforms = A.Compose([
    A.Resize(*target_size, interpolation=cv2.INTER_NEAREST),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7, border_mode=cv2.BORDER_REFLECT),
    A.RandomBrightnessContrast(p=0.5),
    A.ElasticTransform(p=0.2),
    A.GaussianBlur(p=0.3),
    A.GridDistortion(p=0.2),
])

def fix_mask(mask):
    return (mask > 0).astype(np.uint8) * 255


augment_dataset(N, num_to_valid, num_to_test,
                    orig_train_img_dir, orig_train_mask_dir,
                    orig_valid_img_dir, orig_valid_mask_dir,
                    orig_test_img_dir, orig_test_mask_dir,
                    output_base,
                    transforms,
                    function_to_apply_to_masks=fix_mask
                    )

/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imagens totais no dataset original: 165
→ Treino: 116
→ Validação (do treino): 24
→ Teste (do treino): 25

Com N=15, total de imagens geradas no treino será: 1856


/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


→ 0 imagens copiadas da pasta valid original.
→ 0 imagens copiadas da pasta test original.


Copiando valid: 100%|██████████| 24/24 [00:00<00:00, 106.02it/s]


→ 24 imagens copiadas do train para valid.


Copiando test: 100%|██████████| 25/25 [00:00<00:00, 109.78it/s]


→ 25 imagens copiadas do train para test.


Aumentando imagens de treino: 100%|██████████| 116/116 [00:16<00:00,  6.92it/s]


Resumo final:
train_images: 1856 arquivos
train_labels: 1856 arquivos
valid_images: 24 arquivos
valid_labels: 24 arquivos
test_images: 25 arquivos
test_labels: 25 arquivos
